## Take a bunch of runs and re-do them with 

In [1]:
from matplotlib import rc
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import os
import xarray as xr
import subprocess
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
os.chdir("/home/149/ab8992/topographic-NIWs")
basepath = Path.cwd().absolute()

import autolib as al
from dask.distributed import Client
client = Client()
client

Matplotlib is building the font cache; this may take a moment.


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 112.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34177,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 112.00 GiB
Comm: tcp://127.0.0.1:41745,Total threads: 4
Dashboard: /proxy/40305/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:38933,


## Make common forcing

In [2]:
import autolib as al


### COMMON ACROSS ALL EXPERIMENTS. 
##########################################################
nlayers = 20             # number of layers
Lx = 4000                # domain zonal extent [km]
Ly = 4000                # domain meridional extent [km]
H  = 4000                # total fluid's depth in [m]
gridspacing = 2          # in [km]
flat = False
reverse = True
x = np.arange(-Lx/2, Lx/2, gridspacing)
y = np.arange(-Ly/2, Ly/2, gridspacing)
ninterf = nlayers + 1    # number of interfaces
interfaces = np.arange(0,  ninterf)
nx = int(round(Lx/gridspacing))
ny = int(round(Ly/gridspacing))
overrides = ["ADIABATIC = True",
             "RHO_0 = 1027.0",
             f"NJGLOBAL={ny}",
             f"NIGLOBAL={nx}",
             f"LENLON = {Lx}.0",
             f"WESTLON = -{Lx//2}.0",
             f"LENLAT = {Ly}.0",
             f"WESTLAT = -{Ly//2}.0",
             f"NK = {nlayers + 1}"] ## Add one for mixed layer
default_dir = "default_rundir"


## Default values when unperturbed
default_height = 500 # m
default_forcing_latwidth = 100 # km
default_duration = 10 # hours
default_layerdensities = np.linspace(1027,1029,nlayers) ## This is the default stratification. Has be modified to add mixed layer!
default_strength = 1
default_ridge_width = 12.5 # km


## Value s from first pass! 
## Perterbation values
## strat variables 
# strats = [0.25,0.2,0.5,0.75,1,1.25,1.5,2,3,4,0.001,0.01,6]

## Height
heights = [10,20,50,80,120,150,225,275,350,450,600,750,1000,1250,1500,2000] + [650,700]
# heights = [1000]
## Duration
durations = np.linspace(1,10,19) ## Note! Will need to modify the amount of time that model runs for with forcing in this case

## Strength
strengths = [0.2,0.5,0.75,1.25,1.5,2,3,4,10,12,15,20] + [5,6,7,8,9]

## Topog Width
widths = [4,8,16,25,35,50,75,100,125,150,175,200]

## Forcing Width
forcing_widths = [10,18,25,50,75,150,175,200,225,250,275,300,350,400,450,500]



# # Save the common wind forcing
# STRESS_X = al.windstress_gaussian(nx = nx,ny = ny)
# eta_ridge = al.eta_gaussian_hill(
#         nlayers=nlayers,
#         ridge=True,
#         nx = nx,
#         ny = ny,
#         )
   
# al.save_inputdata(
#     x,
#     y,
#     STRESS_X,
#     STRESS_X * 0,
#     eta_ridge,
#     "common",
#     save_wind = False
#     )


In [4]:
expt = al.expt(x,y,nlayers,"common","common",overrides=overrides)
expt.setup()
# expt.run()

In [5]:
f = 1e-4
N = np.sqrt((9.8 * (2 / 1025)) / 4000)

print(N/f)

21.864326664404846


In [10]:
import subprocess
# heights = [10]
for height in heights:
    expt = al.expt(x,y,nlayers,"height",height,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    # path = al.basepath / "rundirs" / expt.variable / f"{expt.variable}_{expt.var_value}"
    expt.run()

RUNNING: height_10
laboratory path:  /scratch/v45/ab8992/mom6
binary path:  /scratch/v45/ab8992/mom6/bin
input path:  /scratch/v45/ab8992/mom6/input
work path:  /scratch/v45/ab8992/mom6/work
archive path:  /scratch/v45/ab8992/mom6/archive
payu: work path already exists.
      Sweeping as --force option is True.
Removing work path /scratch/v45/ab8992/mom6/work/height_10
Setting up mom6
Checking exe and input manifests
Updating full hashes for 1 files in manifests/exe.yaml
Updating full hashes for 4 files in manifests/input.yaml
Creating restart manifest
Updating full hashes for 3 files in manifests/restart.yaml
Writing manifests/input.yaml
Writing manifests/restart.yaml
Writing manifests/exe.yaml
122662055.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/

In [3]:
for duration in durations:
    expt = al.expt(x,y,nlayers,"duration",duration,overrides=overrides)
    expt.make_inputs()
    expt.setup()
    path = al.basepath / "rundirs" / expt.variable / f"{expt.variable}_{expt.var_value}"
    al.overwrite_in_file(
        str(path / "input.nml"),
        "    hours = ",
        f"    hours = {int(2 * duration)}"

    )
    expt.run()
    




In [3]:
for strength in strengths:
    expt = al.expt(x,y,nlayers,"strength",strength,overrides=overrides)
    expt.make_inputs()
    expt.setup()
    path = al.basepath / "rundirs" / expt.variable / f"{expt.variable}_{expt.var_value}"
    expt.run()

RUNNING: strength_0.2
laboratory path:  /scratch/v45/ab8992/mom6
binary path:  /scratch/v45/ab8992/mom6/bin
input path:  /scratch/v45/ab8992/mom6/input
work path:  /scratch/v45/ab8992/mom6/work
archive path:  /scratch/v45/ab8992/mom6/archive
Setting up mom6
Checking exe and input manifests
Updating full hashes for 1 files in manifests/exe.yaml
Updating full hashes for 4 files in manifests/input.yaml
Creating restart manifest
Writing manifests/input.yaml
Writing manifests/restart.yaml
Writing manifests/exe.yaml
124314622.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P v45 -l walltime=21600 -l ncpus=912 -l mem=3648GB -N strength_0.2 -l wd -j n -v PAYU_PATH=/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin,PAYU_FORCE=True,MODUL

In [4]:
for width in widths:
    expt = al.expt(x,y,nlayers,"width",width,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    expt.run()

RUNNING: width_4
laboratory path:  /scratch/v45/ab8992/mom6
binary path:  /scratch/v45/ab8992/mom6/bin
input path:  /scratch/v45/ab8992/mom6/input
work path:  /scratch/v45/ab8992/mom6/work
archive path:  /scratch/v45/ab8992/mom6/archive
payu: work path already exists.
      Sweeping as --force option is True.
Moving log width_4.o122661477
Moving log width_4.e122661477
Removing work path /scratch/v45/ab8992/mom6/work/width_4
Removing symlink /home/149/ab8992/topographic-NIWs/rundirs/width/width_4/work
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
Setting up mom6
Checking exe and input manifests
Updating full hashes for 1 files in manifests/exe.yaml
Updating full hashes for 3 files in manifests/input.yaml
Creating restart manifest
Writing manifests/input.yaml
Writing manifests/restart.yaml
Writing manifests/exe.yaml
122721535.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning:

In [3]:
# forcing_widths = [250,275,300,350,400,450,500]

for forcing_width in forcing_widths:
    expt = al.expt(x,y,nlayers,"forcing_width",forcing_width,overrides=overrides)
    expt.make_inputs()
    expt.setup()
    expt.run()

RUNNING: forcing_width_10
laboratory path:  /scratch/v45/ab8992/mom6
binary path:  /scratch/v45/ab8992/mom6/bin
input path:  /scratch/v45/ab8992/mom6/input
work path:  /scratch/v45/ab8992/mom6/work
archive path:  /scratch/v45/ab8992/mom6/archive
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
Setting up mom6
Checking exe and input manifests
Updating full hashes for 1 files in manifests/exe.yaml
Updating full hashes for 1 files in manifests/input.yaml
Creating restart manifest
Writing manifests/input.yaml
Writing manifests/restart.yaml
Writing manifests/exe.yaml
122813578.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P v45 -l walltime=21600 -l 

In [ ]:
for duration in durations[0:len(durations) - 1]:
    expt = al.expt(x,y,nlayers,"duration",duration,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    # path = al.basepath / "rundirs" / expt.variable / f"{expt.variable}_{expt.var_value}"
    # al.overwrite_in_file(
    #     str(path / "input.nml"),
    #     "    hours = ",
    #     f"    hours = {int(2 * duration)}"

    # )
    print(duration)
    # expt.run()

RUNNING: duration_10.0
laboratory path:  /scratch/v45/ab8992/mom6
binary path:  /scratch/v45/ab8992/mom6/bin
input path:  /scratch/v45/ab8992/mom6/input
work path:  /scratch/v45/ab8992/mom6/work
archive path:  /scratch/v45/ab8992/mom6/archive
Setting up mom6
Checking exe and input manifests
Updating full hashes for 1 files in manifests/exe.yaml
Updating full hashes for 4 files in manifests/input.yaml
Creating restart manifest
Writing manifests/input.yaml
Writing manifests/restart.yaml
Writing manifests/exe.yaml
122637559.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P nm03 -l walltime=21600 -l ncpus=912 -l mem=3648GB -N duration_10.0 -l wd -j n -v PAYU_PATH=/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin,PAYU_FORCE=True,MO

rm: cannot remove '/scratch/v45/ab8992/mom6/work/duration_10.0': Directory not empty
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/payu/subcommands/setup_cmd.py", line 26, in runcmd
    expt.setup(force_archive=force_archive)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/payu/experiment.py", line 390, in setup
    self.sweep()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/payu/experiment.py", line 953, in sweep
    assert rc == 0
AssertionError


laboratory path:  /scratch/v45/ab8992/mom6
binary path:  /scratch/v45/ab8992/mom6/bin
input path:  /scratch/v45/ab8992/mom6/input
work path:  /scratch/v45/ab8992/mom6/work
archive path:  /scratch/v45/ab8992/mom6/archive
payu: work path already exists.
      Sweeping as --force option is True.
Moving log duration_10.0.o122637560
Moving log duration_10.0.e122637560
Removing work path /scratch/v45/ab8992/mom6/work/duration_10.0
122637579.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P nm03 -l walltime=21600 -l ncpus=912 -l mem=3648GB -N duration_10.0 -l wd -j n -v PAYU_PATH=/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin,PAYU_FORCE=True,MODULESHOME=/opt/Modules/v4.3.0,MODULES_CMD=/opt/Modules/v4.3.0/libexec/modulecmd.tcl,MODU

In [6]:
## Modify existing forcing duration data to scale and ensure taux stays constant

# Glob all of the directories in path al.basepath / "inputdir" / "forcing_width*"
dirs = (al.basepath / "inputdir").glob("forcing_width*")

for i in dirs:
    data = xr.open_mfdataset(i / "storm_windstress.nc", chunks="auto", decode_times=False)
    scaling = 100 / int(i.name.split("_")[-1])
    data["STRESS_X"] *= scaling
    data.to_netcdf(i / "storm_windstress.nc", mode="w")

PermissionError: [Errno 13] Permission denied: '/home/149/ab8992/topographic-NIWs/inputdir/forcing_width_450/storm_windstress.nc'